In [1]:
from scipy.io import loadmat
from pathlib import Path

data_path = Path("/home/aleksandernagaj/Milestone/data/")

pytorch_result = loadmat(data_path / "PRW-v16.04.20" / "annotations" / "c1s1_000151.jpg.mat")

pytorch_result

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Feb 23 21:09:28 2016',
 '__version__': '1.0',
 '__globals__': [],
 'box_new': array([[  -2.        ,  193.98137153,  480.39800763,   49.3669103 ,
          149.42472994],
        [   3.        , 1024.64686825,  414.53887689,   48.98488121,
          158.61771058]])}

In [8]:
from annot import (
    Annotation,
    Camera,
)
import re
import cv2
import logging

bbox_annotations = []
for annot in (data_path / "PRW-v16.04.20" / "annotations").glob("*mat"):
    mat = loadmat(annot)
    if "box_new" not in mat:
        logging.warning("bbox not found for %s", annot.name)
        continue

    location = int(re.search(r"(?<=s)\d", annot.name).group(0))
    camera_n = int(re.search(r"(?<=c)\d", annot.name).group(0))
    sequence_n = int(annot.name.split("_")[1].split(".")[0])

    annotations = []
    for bbox in mat["box_new"]:
        annotations.append(Annotation(
            id_= int(round(bbox[0])),
            bbox=tuple(int(round(b)) for b in bbox[1:]),
            copy_id=True,
        ))

    camera = Camera(
        image=cv2.imread(
            str(
                data_path / "PRW-v16.04.20" / "frames" / 
                ".".join(annot.name.split(".")[:2])
            )
        ),
        location=location,
        n=camera_n,
        sequence_n=sequence_n,
        annotations=annotations
    )
    cropped = camera.crop_bbox()
    for crop in cropped:
        try:
            crop.export_to_reid(data_path / "PRW-v16.04.20" / "exported")
        except:
            pass

In [1]:
from annot import prepare_train_test_set, create_query_from_gallery
from pathlib import Path

data_path = Path("/home/aleksandernagaj/Milestone/data/PRW-v16.04.20")

# prepare_train_test_set(
#     src=data_path / "exported",
#     dst=data_path / "pytorch",
#     val=True,
# )
create_query_from_gallery(
    gallery_path=data_path / "pytorch" / "gallery",
    query_path=data_path / "pytorch" / "query",
    noise_dir=data_path / "pytorch" / "gallery" / "00-2",
    backup=True,
)

In [ ]:
data_path = Path("/home/aleksandernagaj/Milestone/data/Milestone")

prepare_train_test_set(
    src=data_path / "exported",
    dst=data_path / "pytorch",
    val=False,
    train_size=0,
    test_size=1,
)